#1. Funciones auxiliares y lectura de datos


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

def leer_rango_excel_a_array(ruta_archivo, hoja_nombre=0, columnas_a_leer="", filas_a_leer=1):
    """
    Abre un archivo Excel y extrae un rango de celdas específico
    (D3 a BO42) y lo guarda en un array (lista de listas).
"""
    print(f"Intentando leer el archivo: {ruta_archivo}...")

    try:
        # 1. Filas:
        # Queremos empezar en la Fila 3.
        # Las filas en pandas se indexan desde 0.
        # Fila 1 = índice 0, Fila 2 = índice 1, Fila 3 = índice 2.
        # Así que saltamos las 2 primeras filas.
        filas_a_saltar = 3  # (Saltamos Fila 1 :texto Fila 2 : calibracion y Fila 3: checkeo salon vs dormitorio)


        #filas_a_leer = 40
        # Queremos leer desde la fila 3 hasta la 42 (inclusive).
        # Número de filas = 42 - 3 + 1 = 40 filas.

        # --- Lectura del archivo ---

        # Usamos 'header=None' porque el rango (D3) no es un encabezado,
        # es el inicio de los datos.
        df_rango = pd.read_excel(
            ruta_archivo,
            sheet_name= hoja_nombre, #Segunda hoja, Espectro Total
            header=None,             # Trata la primera fila leída (Fila 3) como datos
            skiprows=filas_a_saltar, # Salta las 2 primeras filas
            usecols=columnas_a_leer, # Lee solo el rango de columnas D-BO
            nrows=filas_a_leer       # Lee exactamente 40 filas
        )

        # Convertir el DataFrame de pandas a un array (lista de listas)
        # .values obtiene los datos como un array de NumPy
        # .tolist() lo convierte a una lista estándar de Python
        array_valores = df_rango.values.tolist()

        # Verificación
        if array_valores:
            num_filas = len(array_valores)
            num_cols = len(array_valores[0]) # Longitud de la primera fila
            print(f"Éxito: Se leyó un array de {num_filas} filas x {num_cols} columnas.")
        else:
            print("Advertencia: El rango D3:BO42 parece estar vacío.")

        return array_valores

    except FileNotFoundError:
        print(f"Error: No se pudo encontrar el archivo en la ruta: '{ruta_archivo}'.")
        return None
    except Exception as e:
        print(f"Ocurrió un error inesperado al leer el rango: {e}")
        return None

def separar_filas_en_matrices(datos_array):
    """
    Toma una lista de listas (matriz) y la separa en dos matrices:
    - Una con las filas de índice par (0, 2, 4...)
    - Otra con las filas de índice impar (1, 3, 5...)
    """

    # Listas para guardar las *filas* (que son listas)
    matriz_filas_impares_excel = []
    matriz_filas_pares_excel = []

    for i, fila in enumerate(datos_array):

        # i % 2 == 0 significa índice 0, 2, 4... (Filas Excel 3, 5, 7...)
        if i % 2 == 0:
            # --- CAMBIO CLAVE ---
            # Añade la 'fila' (que es una lista) como un nuevo elemento
            matriz_filas_impares_excel.append(fila)

        # else significa índice 1, 3, 5... (Filas Excel 4, 6, 8...)
        else:
            # --- CAMBIO CLAVE ---
            matriz_filas_pares_excel.append(fila)

    return matriz_filas_impares_excel, matriz_filas_pares_excel


In [2]:
#LO PRIMERO: ABRIR LOS ARCHIVOS

archivo_S1= "P3_G7_S1_Datos.xlsx"
archivo_S2= "P3_G7_S2_Datos.xlsx"

datos_sonometro_1 = leer_rango_excel_a_array(archivo_S1, "Banda Ancha Total", "U" , 24)
datos_sonometro_2 = leer_rango_excel_a_array(archivo_S2, "Banda Ancha Total", "U" , 48)

#SEPARAR POR PUNTOS A B Y C Y LUEGO RUIDOS DE FONDO

# Filtra los índices 3, 4, 5 de cada bloque de 6
datos_A = [x for i, x in enumerate(datos_sonometro_2) if i % 6 >= 3]

# Filtra los índices 0, 1, 2 de cada bloque de 6
datos_C = [x for i, x in enumerate(datos_sonometro_2) if i % 6 < 3]

ruido_A = np.array(datos_A[:3]) #QUITAR LAS TRES PRIMERAS POSICIONES QUE SON DE RUIDO DE FONDO
medidas_A = np.array(datos_A[3:])

ruido_B = np.array(datos_sonometro_1[:3]) #RUIDO EN B
medidas_B = np.array(datos_sonometro_1[3:]) #MEDIDAS EN B

ruido_C = np.array(datos_C[:3])
medidas_C = np.array(datos_C[3:])

#print(ruido_A, ruido_B, ruido_C) PARA COMPROBAR QUE FUNCIONE

#CORRECION RUIDO DE FONDO: PROMEDIADO DEL RUIDO DE FONDO ENERGETICO

ruido_A_promed = np.round(10 * np.log10((1/len(ruido_A)) * np.sum(10**(0.1*ruido_A))), 1)
ruido_B_promed = np.round(10 * np.log10((1/len(ruido_B)) * np.sum(10**(0.1*ruido_B))), 1)
ruido_C_promed = np.round(10 * np.log10((1/len(ruido_C)) * np.sum(10**(0.1*ruido_C))), 1)

#print(ruido_A_promed,ruido_B_promed, ruido_C_promed) #TUDO VA BEM

#print(len(medidas_A))
#AHORA VOY A SEPARAR LAS FASES

fases_A = np.split(medidas_A,7) #En este array están las medidas de cada fase, SIN PROMEDIAR
fases_B = np.split(medidas_B,7) #En este array están las medidas de cada fase, SIN PROMEDIAR
fases_C = np.split(medidas_C,7) #En este array están las medidas de cada fase, SIN PROMEDIAR


#print(fases_A[0]) Te devuelve los tres valores de la FASE 1 en la posición A

fases_A_promed = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_A
])

fases_B_promed = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_B
])

fases_C_promed = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_C
])

#TODO: COMPROBAR EL CRITERIO EN CADA FASE DE QUE EL RUIDO DE FONDO SEA 3dB INFERIOR A LA MEDIDA DE LA FASE. SE REALIZA SOBRE EL PROMEDIADO DE LAS TRES MEDIDAS EN CADA PUNTO
#EJEMPLO : FASES_A_PROMED[0]-RUIDO_A_PROMED PARA COMPROBAR EN LA FASE 1
#A INVESTIGAR ANTES: ES POSIBLE QUE HAYA QUE HACER ANTES EL PROMEDIADO DE LOS PUNTOS ABC PARA COMPROBAR ESTO CONTRA EL RUIDO ABC? LA INTUICION ME DICE QUE NO, QUE ES PUNTO A PUNTO

#Comprobación de cada punto, cada fase si cumple el criterio de los 3dB

ruido_promed = np.round(10 * np.log10((1/3) * ((10**(0.1*ruido_A_promed)) + (10**(0.1*ruido_B_promed)) + (10**(0.1*ruido_C_promed)))), 1)
fases_promed = np.round(10 * np.log10((1/3) * ((10**(0.1*fases_A_promed)) + (10**(0.1*fases_B_promed)) + (10**(0.1*fases_C_promed)))), 1)

diferencias = fases_promed - ruido_promed
print(diferencias)

#LAS FASES QUE NO CUMPLEN SON LAS 5, 6 Y 7. EN ELLAS NO SE PUEDE CORREGIR POR RUIDO DE FONDO Y LOS RESULTADOS SE DARÁN COMO COTAS

fases_1234 = fases_promed[:4]
fases_567 = fases_promed[4:7]

fases_1234_corregidas = np.round(10 * np.log10(((10**(0.1*fases_1234)) - (10**(0.1*ruido_promed)))), 1)

La_fases =  np.concatenate((fases_1234_corregidas , fases_567) , axis = 0)

print(fases_promed, La_fases)

Intentando leer el archivo: P3_G7_S1_Datos.xlsx...
Éxito: Se leyó un array de 24 filas x 1 columnas.
Intentando leer el archivo: P3_G7_S2_Datos.xlsx...
Éxito: Se leyó un array de 48 filas x 1 columnas.
[3.9 6.3 6.6 3.7 2.  0.5 0.9]
[43.3 45.7 46.  43.1 41.4 39.9 40.3] [41.  44.5 44.9 40.7 41.4 39.9 40.3]


c:\Users\34666\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\34666\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
#Importamos la columna correspondiente a LCeq,T

datos_sonometro_1_LCeq = leer_rango_excel_a_array(archivo_S1, "Banda Ancha Total", "X" , 24)
datos_sonometro_2_LCeq = leer_rango_excel_a_array(archivo_S2, "Banda Ancha Total", "X" , 48)



datos_A_LCeq = [x for i, x in enumerate(datos_sonometro_2_LCeq) if i % 6 >= 3]


datos_C_LCeq = [x for i, x in enumerate(datos_sonometro_2_LCeq) if i % 6 < 3]

ruido_A_LCeq = np.array(datos_A_LCeq[:3])
medidas_A_LCeq = np.array(datos_A_LCeq[3:])

ruido_B_LCeq = np.array(datos_sonometro_1_LCeq[:3])
medidas_B_LCeq = np.array(datos_sonometro_1_LCeq[3:])

ruido_C_LCeq = np.array(datos_C_LCeq[:3])
medidas_C_LCeq = np.array(datos_C_LCeq[3:])


ruido_A_LCeq_promed = np.round(10 * np.log10((1/len(ruido_A_LCeq)) * np.sum(10**(0.1*ruido_A_LCeq))), 1)
ruido_B_LCeq_promed = np.round(10 * np.log10((1/len(ruido_B_LCeq)) * np.sum(10**(0.1*ruido_B_LCeq))), 1)
ruido_C_LCeq_promed = np.round(10 * np.log10((1/len(ruido_C_LCeq)) * np.sum(10**(0.1*ruido_C_LCeq))), 1)


fases_A_LCeq = np.split(medidas_A_LCeq,7)
fases_B_LCeq = np.split(medidas_B_LCeq,7)
fases_C_LCeq = np.split(medidas_C_LCeq,7)


fases_A_promed_LCeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_A_LCeq
])

fases_B_promed_LCeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_B_LCeq
])

fases_C_promed_LCeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_C_LCeq
])



ruido_promed_LCeq = np.round(10 * np.log10((1/3) * ((10**(0.1*ruido_A_LCeq_promed)) + (10**(0.1*ruido_B_LCeq_promed)) + (10**(0.1*ruido_C_LCeq_promed)))), 1)
fases_promed_LCeq = np.round(10 * np.log10((1/3) * ((10**(0.1*fases_A_promed_LCeq)) + (10**(0.1*fases_B_promed_LCeq)) + (10**(0.1*fases_C_promed_LCeq)))), 1)

print(ruido_promed_LCeq)
print(fases_promed_LCeq)

diferencias_LCeq = fases_promed_LCeq - ruido_promed_LCeq
print("Valores de LCeq - LAeq para cada fase de ruido: ",diferencias_LCeq) #NO CUMPLE NINGUNA, FULL COTAS

LCeq_fases = fases_promed_LCeq
print("Valores de LCeq con correccion de RF para cada fase de ruido: ",LCeq_fases)

Lf =  LCeq_fases - La_fases
print("Valores de Lf para cada fase de ruido: ",Lf)

Kf = [6,6,6,6,6,6,6] #TODOS LOS VALORES SUPERAN 15, POR TANTO TODAS LAS Kf SON 6 dB
print("Valores de Kf para cada fase de ruido: ",Kf)



Intentando leer el archivo: P3_G7_S1_Datos.xlsx...
Éxito: Se leyó un array de 24 filas x 1 columnas.
Intentando leer el archivo: P3_G7_S2_Datos.xlsx...
Éxito: Se leyó un array de 48 filas x 1 columnas.
60.1
[60.2 60.8 60.5 59.9 60.2 60.  59.8]
Valores de LCeq - LAeq para cada fase de ruido:  [ 0.1  0.7  0.4 -0.2  0.1 -0.1 -0.3]
Valores de LCeq con correccion de RF para cada fase de ruido:  [60.2 60.8 60.5 59.9 60.2 60.  59.8]
Valores de Lf para cada fase de ruido:  [19.2 16.3 15.6 19.2 18.8 20.1 19.5]
Valores de Kf para cada fase de ruido:  [6, 6, 6, 6, 6, 6, 6]


c:\Users\34666\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\34666\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
#Importamos la columna correspondiente a LAIeq,T

datos_sonometro_1_LAIeq = leer_rango_excel_a_array(archivo_S1, "Banda Ancha Total", "S" , 24)
datos_sonometro_2_LAIeq = leer_rango_excel_a_array(archivo_S2, "Banda Ancha Total", "S" , 48)



datos_A_LAIeq = [x for i, x in enumerate(datos_sonometro_2_LAIeq) if i % 6 >= 3]


datos_C_LAIeq = [x for i, x in enumerate(datos_sonometro_2_LAIeq) if i % 6 < 3]

ruido_A_LAIeq = np.array(datos_A_LAIeq[:3])
medidas_A_LAIeq = np.array(datos_A_LAIeq[3:])

ruido_B_LAIeq = np.array(datos_sonometro_1_LAIeq[:3])
medidas_B_LAIeq = np.array(datos_sonometro_1_LAIeq[3:])

ruido_C_LAIeq = np.array(datos_C_LAIeq[:3])
medidas_C_LAIeq = np.array(datos_C_LAIeq[3:])


ruido_A_LAIeq_promed = np.round(10 * np.log10((1/len(ruido_A_LAIeq)) * np.sum(10**(0.1*ruido_A_LAIeq))), 1)
ruido_B_LAIeq_promed = np.round(10 * np.log10((1/len(ruido_B_LAIeq)) * np.sum(10**(0.1*ruido_B_LAIeq))), 1)
ruido_C_LAIeq_promed = np.round(10 * np.log10((1/len(ruido_C_LAIeq)) * np.sum(10**(0.1*ruido_C_LAIeq))), 1)


fases_A_LAIeq = np.split(medidas_A_LAIeq,7)
fases_B_LAIeq = np.split(medidas_B_LAIeq,7)
fases_C_LAIeq = np.split(medidas_C_LAIeq,7)


fases_A_promed_LAIeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_A_LAIeq
])

fases_B_promed_LAIeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_B_LAIeq
])

fases_C_promed_LAIeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_C_LAIeq
])


ruido_promed_LAIeq = np.round(10 * np.log10((1/3) * ((10**(0.1*ruido_A_LAIeq_promed)) + (10**(0.1*ruido_B_LAIeq_promed)) + (10**(0.1*ruido_C_LAIeq_promed)))), 1)
fases_promed_LAIeq = np.round(10 * np.log10((1/3) * ((10**(0.1*fases_A_promed_LAIeq)) + (10**(0.1*fases_B_promed_LAIeq)) + (10**(0.1*fases_C_promed_LAIeq)))), 1)

diferencias_LAIeq = fases_promed_LAIeq - ruido_promed_LAIeq

print("Valores de LAIeq - LAIeqRF para cada fase de ruido: ",diferencias_LAIeq) #CUMPLEN 1,2,3; NO CUMPLEN 4,5,6,7

fases_123 = fases_promed[:3]
fases_4567 = fases_promed[3:7]

fases_123_corregidas = np.round(10 * np.log10(((10**(0.1*fases_123)) - (10**(0.1*ruido_promed_LAIeq)))), 1)

LAIeq_fases =  np.concatenate((fases_123_corregidas , fases_4567) , axis = 0)
print("Valores de LAIeq con correccion de RF para cada fase de ruido: ",LAIeq_fases)

Li =  LAIeq_fases - La_fases
print("Valores de Li para cada fase de ruido: ",Li)

Ki = [0,0,0,0,0,0,0] #LOS QUE VALEN 6 ES POR LAS COTAS, EL RESTO ESTÁN TODOS POR DEBAJO DE 10, LUEGO SU Ki VALE 0
print("Valores de Ki para cada fase de ruido: ",Ki)

#TODO QUE ALGUIEN REVISE LO DE LAS COTAS, YO LE VEO SENTIDO PERO ECHADLE UN OJO POR CONTRASTAR

Intentando leer el archivo: P3_G7_S1_Datos.xlsx...
Éxito: Se leyó un array de 24 filas x 1 columnas.
Intentando leer el archivo: P3_G7_S2_Datos.xlsx...
Éxito: Se leyó un array de 48 filas x 1 columnas.
Valores de LAIeq - LAeq para cada fase de ruido:  [ 5.2 13.3 13.4  2.9  0.9 -0.8 -0.2]
Valores de LAIeq con correccion de RF para cada fase de ruido:  [37.1 43.2 43.7 43.1 41.4 39.9 40.3]
Valores de Li para cada fase de ruido:  [-3.9 -1.3 -1.2  2.4  0.   0.   0. ]
Valores de Ki para cada fase de ruido:  [0, 0, 0, 0, 6, 6, 6]


c:\Users\34666\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\34666\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
Espectro_total_s1 = leer_rango_excel_a_array(archivo_S1, "Espectro Total", "EK:FG" , 24)
Espectro_total_s2 = leer_rango_excel_a_array(archivo_S2, "Espectro Total", "D:FG" , 48)

datos_A_espectro = [x for i, x in enumerate(Espectro_total_s2) if i % 6 >= 3]
datos_C_espectro = [x for i, x in enumerate(Espectro_total_s2) if i % 6 < 3]

ruido_A_espectro = np.array(datos_A_espectro[:3])
medidas_A_espectro = np.array(datos_A_espectro[3:])

ruido_B_espectro = np.array(Espectro_total_s1[:3])
medidas_B_espectro = np.array(Espectro_total_s1[3:])

ruido_C_espectro = np.array(datos_C_espectro[:3])
medidas_C_espectro = np.array(datos_C_espectro[3:])

#TODO HAY QUE CALCULAR LT PARA SACAR LAS Kt
#DUDA: ¿COMO CARAJO ESTIMAMOS TENIENDO EN CUENTA LO DE LAS COTAS?

Intentando leer el archivo: P3_G7_S1_Datos.xlsx...
Éxito: Se leyó un array de 24 filas x 160 columnas.
[[54.75 51.49 56.46 60.9  52.96 46.71 62.06 53.62 48.79 44.14 50.16 50.95
  45.96 36.54 36.31 31.9  30.68 32.17 29.07 27.45 25.38 24.03 21.99 21.68
  22.53 22.88 20.56 20.16 20.22 18.64 18.26 19.05 54.75 51.49 56.46 59.68
  51.73 46.16 59.57 51.76 46.78 41.1  47.32 48.39 43.53 34.4  34.19 29.66
  29.07 30.75 27.63 24.94 23.76 21.56 19.98 19.89 19.42 19.46 18.55 18.07
  17.94 16.83 16.51 17.64 51.84 46.02 54.19 53.94 47.6  43.41 53.4  47.48
  41.73 36.61 42.58 45.21 39.88 31.57 30.38 27.28 26.51 28.19 25.81 22.67
  21.64 18.19 15.32 13.11 11.94 12.35 11.57 11.17 11.5  11.69 12.61 14.86
  51.84 46.02 54.19 55.23 47.91 44.33 56.17 48.6  43.79 39.58 45.27 45.36
  42.36 32.99 31.97 28.41 27.75 29.08 26.94 24.15 22.57 19.69 17.67 17.38
  16.95 17.1  16.06 15.53 15.2  14.06 14.03 15.66 53.8  50.1  55.49 56.57
  49.98 45.64 58.52 50.81 45.41 40.32 46.65 47.92 42.86 33.84 33.69 29.38
  28.54 3

c:\Users\34666\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
